In [53]:
library(tidyverse)
bugbank_data_dir = "/well/bag/clme1992/bugbank_data/"
ukb_data_dir = "/well/bag/wilson/ukb/"
gwas_wrkdir = "/well/bag/clme1992/saige_pipe_test"


In [2]:
# At each step of QC, track the change in the infection cases

resolve.symlink <- function(x) {
    y <- Sys.readlink(x)
    if (any(is.na(y))) {
        stop("Could not resolve symlink ", x)
    }
    ifelse(y == "", x, file.path(dirname(x), y))
}

# configuration file
source("~/.saige_pipe.config")

lg <- list()
lg$bd_RDdata_file <- paste0(config$ukb.derived.dir, "/ukb41482.ukb41376.fields.RData")
lg$bd_not_lost2followup_file <- paste0(config$ukb.derived.dir, "/ukb41482.English-not-lost-to-followup-8-April-2020.txt")
lg$bed_sample_qc_file <- paste0(config$ukbdir, "/v2/qc/ukb_sqc_v2.txt")
lg$withdrawn_eid_file <- paste0(config$bbdatadir, "/w53100_2023-04-25.csv")
# Pre-computed eids for the bed-format genotypes
lg$bed_eid_file <- paste0(config$ukb.derived.dir, "/analysis.bed.eids.txt")
# Individuals with first degree relatives
lg$remrels_file <- paste0(config$ukb.derived.dir, "/ukb41482.English-remove-first-degree-relatives.eids.txt")
# panUKB ancestral files
lg$pan_ukb_file <- paste0(config$panukb.dir, "/Files for retman/all_pops_non_eur_pruned_within_pop_pc_covs.tsv")
lg$pan_ukb_bridge_file <- paste0(config$panukb.dir, "/ukb53100bridge31063.txt")

### load input files ###
system.time(load(lg$bd_RDdata_file))
all_eids <- bd[, "f.eid"]
bd_not_lost2followup <- scan(lg$bd_not_lost2followup_file, what = "logical") == "TRUE"
withdrawn_eid <- scan(lg$withdrawn_eid_file)
# Sample QC
bed_sample_qc <- read.csv(lg$bed_sample_qc_file, sep = " ")
# The corresponding eids
bed_eid <- scan(lg$bed_eid_file)
# Convert to bd_eid order
sample_qc <- bed_sample_qc[match(all_eids, bed_eid), ]
# Close (first degree) relatives
remrels <- scan(lg$remrels_file)
# load and match the panukb data
panukb <- read.csv(lg$pan_ukb_file, sep = "\t")[, c("s", "pop")]
panukb_bridge <- read.csv(lg$pan_ukb_bridge_file, sep = " ", header = F)
bridge_matched <- panukb_bridge[match(panukb$s, panukb_bridge[, 2]), ]
panukb$eid <- bridge_matched[, 1]
panukb_matched <- panukb[match(all_eids, panukb$eid), ]

### assign assessment centre data ###
f.assesscentre <- "f.54.0.0"
assess_centre_England <- c(
    11012, 11021, 11011, 11008, 11024, 11020, 11018, 11010, 11016, 
    11001, 11017, 11009, 11013, 11002, 11007, 11014, 10003, 11006, 
    11025, 11026, 11027, 11028 
)
f_assesscentre <- "f.54.0.0"

   user  system elapsed 
 15.271   1.696  17.079 

In [7]:
# get summary for SGSS
sgss_file = paste0(bugbank_data_dir, "ukb_sgss_extract_refined.csv")
sgss = read.csv(sgss_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)

# load HES
start_time = Sys.time()
hes_file = paste0(ukb_data_dir, "hes/hesin_diag.latest.txt.gz")
hes = read.table(hes_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)
end_time = Sys.time()
print(paste0("Time taken to load hes: ", end_time - start_time))

[1] "Time taken to load hes: 20.7608504295349"


In [8]:
# load icd10 data
icd10_desc_file <- paste0(bugbank_data_dir, "pathogen_to_unique_icd10.tsv")
icd10_desc <- read.table(icd10_desc_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)

# subset hes to only infection related
infect_icd10_codes <- unique(unlist(strsplit(icd10_desc$icd10, split = ",")))
hes_infect <- hes[hes$diag_icd10 %in% infect_icd10_codes, ]

In [32]:
length(bd$f.eid)
length(unique(bd$f.eid))

length(bd$f.eid[filter_sgss])
length(unique(bd$f.eid[filter_sgss]))

[1] 502505

[1] 502505

[1] 114529

[1] 110987

In [43]:
filter_lst <- list()
filter <- rep(T, nrow(bd))
filter_lst[[1]] <- filter
names(filter_lst)[1] <- "all"

filter <- filter & bd_not_lost2followup
filter <- filter & !(all_eids %in% withdrawn_eid)
filter_lst[[2]] <- filter
names(filter_lst)[2] <- "not lost to followup"

filter <- filter & sample_qc$putative.sex.chromosome.aneuploidy == 0
filter_lst[[3]] <- filter
names(filter_lst)[3] <- "no aneuploidy in sex chromosome"

filter <- filter & sample_qc$Submitted.Gender == sample_qc$Inferred.Gender
filter_lst[[4]] <- filter
names(filter_lst)[4] <- "reported sex matches genetic sex"

filter <- filter & sample_qc$het.missing.outliers == 0
filter_lst[[5]] <- filter
names(filter_lst)[5] <- "no het missing outliers"

filter <- filter & sample_qc$excluded.from.kinship.inference == 0
filter_lst[[6]] <- filter
names(filter_lst)[6] <- "not excluded from kinship inference"

filter <- filter & sample_qc$excess.relatives == 0
filter_lst[[7]] <- filter
names(filter_lst)[7] <- "no excess relatives"

filter <- filter & sample_qc$in.Phasing.Input.chr1_22 == 1 & 
    sample_qc$in.Phasing.Input.chrX == 1 &
    sample_qc$in.Phasing.Input.chrXY == 1
filter_lst[[8]] <- filter
names(filter_lst)[8] <- "in phasing input"

filter <- filter & all_eids %in% panukb_matched$eid[which(panukb_matched$pop == "EUR")]
filter_lst[[9]] <- filter
names(filter_lst)[9] <- "in panukb EUR"

filter <- filter & bd[, f_assesscentre] %in% assess_centre_England
filter_lst[[10]] <- filter
names(filter_lst)[10] <- "in assessment centre England"



In [50]:
sgss_filter <- sgss
hes_filter <- hes_infect
ukb_filter <- list()
for (i in 1:length(filter_lst)) {
    filter <- filter_lst[[i]]
    eids_filtered <- unique(all_eids[filter])
    sgss_filter <- sgss_filter[sgss_filter$UKB_EID %in% eids_filtered, ]
    hes_filter <- hes_filter[hes_filter$eid %in% eids_filtered, ]
    sgss_cnt <- length(unique(sgss_filter$UKB_EID))
    hes_cnt <- length(unique(hes_filter$eid))
    ukb_filter[[i]] <- c(length(eids_filtered), sgss_cnt, hes_cnt)
}
ukb_filter <- do.call(rbind, ukb_filter)
rownames(ukb_filter) <- names(filter_lst)

ukb_filter

all,502505,114736,69900
not lost to followup,426755,108226,56142
no aneuploidy in sex chromosome,414121,104782,54100
reported sex matches genetic sex,413957,104733,54067
no het missing outliers,413123,104500,53948
not excluded from kinship inference,413115,104498,53948
no excess relatives,412956,104456,53924
in phasing input,412721,104385,53878
in panukb EUR,359929,92639,46432
in assessment centre England,359929,92639,46432


### Pathogen filtering

In [54]:
filter_lst <- list()
filter <- rep(T, nrow(sgss))

filter_lst[[1]] <- filter
names(filter_lst)[1] <- "all"

sgss_tax_file <- paste0(bugbank_data_dir, "bb_pathogen_taxonomy_13032023.tsv")
sgss_tax <- read.table(sgss_tax_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)
sgss_tax$origin_name[which(!is.na(sgss_tax$origin_name))] <- 
species_org <- sgss_tax$origin_name[which(!is.na(sgss_tax$species))]
filter_lst[[2]] <- filter & sgss$ORGANISM_SPECIES_NAME %in% species_org
names(filter_lst)[2] <- "species level pathogen"

sgss_pheno_rds_file <- paste0(gwas_wrkdir, "/log.ukb41482.bd.gwasdata.05062023_sgss_species.rds")
sgss_pheno_rds <- readRDS(sgss_pheno_rds_file)
sgss_pheno_rds$

filter_lst[[3]] <- filter 
# number of records
print(paste0("Number of records: ", nrow(sgss)))
# number of unique individuals
print(paste0("Number of unique individuals: ", length(unique(sgss$UKB_EID))))
# number of pathogen labels
print(paste0("Number of pathogen labels: ", length(unique(sgss$ORGANISM_SPECIES_NAME))))

Warning message in sgss_tax$origin_name[which(!is.na(sgss_tax$origin_name))] <- species_org <- sgss_tax$origin_name[which(!is.na(sgss_tax$species))]:
"number of items to replace is not a multiple of replacement length"


[1] "Number of records: 350699"


[1] "Number of unique individuals: 114737"
[1] "Number of pathogen labels: 641"


In [59]:
sgss_pheno_rds$cases.counts[3]

Escherichia_coli.species.MSU 
                           0

In [19]:
# load HES
start_time = Sys.time()
hes_file = paste0(ukb_data_dir, "hes/hes_diag.latest.txt.gz")
hes = read.table(hes_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)
end_time = Sys.time()
print(paste0("Time taken to load hes: ", end_time - start_time))

[1] "Time taken to load HESIN: 17.3452785015106"


In [12]:
# load icd10 data
icd10_desc_file <- paste0(bugbank_data_dir, "pathogen_to_unique_icd10.tsv")
icd10_desc <- read.table(icd10_desc_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)

In [33]:
# subset hes to only infection related
infect_icd10_codes <- unique(unlist(strsplit(icd10_desc$icd10, split = ",")))
hes_infect <- hes[hes$diag_icd10 %in% infect_icd10_codes, ]

# number of records
print(paste0("Number of infection related records: ", nrow(hes_infect)))

# number of unique individuals
print(paste0("Number of unique individuals: ", length(unique(hes_infect$eid))))

# number of unique icd10 codes
print(paste0("Number of unique icd10 codes: ", length(unique(hes_infect$diag_icd10))))

# number of species level pathogens
# create a dictionary of ICD-10 to pathogen mapping
icd_to_pathogen = list()
for (i in 1:nrow(icd10_desc)) {
  cur_icd10s = unlist(strsplit(icd10_desc$icd10[i], ","))
  for (icd10 in cur_icd10s) {
    icd_to_pathogen[[icd10]] = c(icd10_desc$org_name[i], icd10_desc$tax_lev[i])
  }
}
# map hes icd10 codes to pathogen
hes_infect$org_name = map_chr(hes_infect$diag_icd10, function(x) icd_to_pathogen[[x]][1])
print(paste0("Number of unique pathogens: ", length(unique(hes_infect$org_name))))

# number of species level pathogen
hes_infect$tax_lev = map_chr(hes_infect$diag_icd10, function(x) icd_to_pathogen[[x]][2])
hes_infect_species = hes_infect[hes_infect$tax_lev == "species", ]
print(x = paste0("Number of unique species level pathogens: ", length(unique(hes_infect_species$org_name))))


[1] "Number of infection related records: 188788"
[1] "Number of unique individuals: 69900"
[1] "Number of unique icd10 codes: 480"
[1] "Number of unique pathogens: 155"
[1] "Number of unique species level pathogens: 88"


In [32]:
head(hes_infect_species)

,eid,ins_index,arr_index,level,diag_icd9,diag_icd9_nb,diag_icd10,diag_icd10_nb,org_name,tax_lev
,<int>,<int>,<int>,<int>,<chr>,<lgl>,<chr>,<int>,<chr>,<chr>
2750,1000906,6,5,2,,NA,B962,NA,Escherichia coli,species
4438,1001488,10,1,2,,NA,B961,NA,Klebsiella pneumoniae,species
4455,1001488,11,1,2,,NA,B961,NA,Klebsiella pneumoniae,species
4702,1001603,2,2,2,,NA,B962,NA,Escherichia coli,species
5375,1001970,8,4,2,,NA,B962,NA,Escherichia coli,species
5712,1002128,7,3,2,,NA,B171,NA,Hepacivirus C,species
